In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import numpy as np
import time
import copy


from torchvision.transforms import transforms
from PIL import Image
import torchvision.models as torchvision_models

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

device: cuda


In [3]:
torch.manual_seed(42)

In [ ]:
train_df = pd.read_csv("D:\\Work\wsl\\ML-DS\\fashion-mnist\\fashion-mnist_train.csv").head(15000)
test_df = pd.read_csv("D:\\Work\wsl\\ML-DS\\fashion-mnist\\fashion-mnist_test.csv").head(1500)

In [5]:
print("train_df.count:",train_df.count())
print("test_df.count:", test_df.count())

train_df.count: label       15000
pixel1      15000
pixel2      15000
pixel3      15000
pixel4      15000
            ...  
pixel780    15000
pixel781    15000
pixel782    15000
pixel783    15000
pixel784    15000
Length: 785, dtype: int64
test_df.count: label       1500
pixel1      1500
pixel2      1500
pixel3      1500
pixel4      1500
            ... 
pixel780    1500
pixel781    1500
pixel782    1500
pixel783    1500
pixel784    1500
Length: 785, dtype: int64


In [6]:
train_df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
test_df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,9,8,...,103,87,56,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,34,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,14,53,99,...,0,0,0,0,63,53,31,0,0,0
3,2,0,0,0,0,0,0,0,0,0,...,137,126,140,0,133,224,222,56,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# Converting to Numpy
X_train = train_df.iloc[:,1:].values
y_train = train_df.iloc[:,0].values

X_test = test_df.iloc[:,1:].values
y_test = test_df.iloc[:,0].values

In [9]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)  # Fit and transform on training set
X_test_scaled = scaler.transform(X_test) 

In [10]:
#https://docs.pytorch.org/vision/main/models/generated/torchvision.models.vgg16.html
"""
The inference transforms are available at VGG16_Weights.IMAGENET1K_V1.transforms and perform the following preprocessing operations: 
Accepts PIL.Image, batched (B, C, H, W) and single (C, H, W) image torch.Tensor objects. 
The images are resized to resize_size=[256] using interpolation=InterpolationMode.BILINEAR, followed by a central crop of crop_size=[224]. 
Finally the values are first rescaled to [0.0, 1.0] and then normalized using mean=[0.485, 0.456, 0.406] and std=[0.229, 0.224, 0.225].
"""
create_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [11]:
class CustomDataset(Dataset):
    def __init__(self, features, labels, transforms):
        super().__init__()
        self.features = features
        self.labels = labels
        self.transform = transforms

    def __len__(self):
        return len(self.features)
    def __getitem__(self, index):
        image = self.features[index].reshape(28, 28)
        image = image.astype(np.uint8)
        image = np.stack([image]*3, axis=-1)
        image = Image.fromarray(image)
        image = self.transform(image)

        return image, torch.tensor(self.labels[index], dtype=torch.long)

In [12]:
train_dataset = CustomDataset(X_train_scaled, y_train, transforms=create_transform)
test_dataset = CustomDataset(X_test_scaled, y_test, transforms=create_transform)

In [13]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory=True)
num_cls = len(np.unique(y_train))

In [14]:
torch.hub.set_dir('D:\\Work\\TORCH_HOME')
vgg16 = torchvision_models.vgg16(pretrained=True)

d:\Work\miniconda_env\hf-pytorch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Work\miniconda_env\hf-pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [15]:
vgg16

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [16]:
for param in vgg16.features.parameters():
    param.requires_grad = False

In [17]:
vgg16.classifier = nn.Sequential( 
                nn.Linear(25088, 4096),
                nn.ReLU(),
                nn.Dropout(.5),
                nn.BatchNorm1d(4096),
                nn.Linear(4096, 1024),
                nn.ReLU(),
                nn.Dropout(.5),
                nn.BatchNorm1d(1024),
                nn.Linear(1024, 512),
                nn.ReLU(),
                nn.Dropout(.5),
                nn.BatchNorm1d(512),
                nn.Linear(512, num_cls))

In [18]:
vgg16 = vgg16.to(device=device)
vgg16

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [19]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.best_model_state = None

    def __call__(self, val_loss, model):
        if self.best_score is None:
            self.best_score = val_loss
            self.best_model_state = copy.deepcopy(model.state_dict())
        elif val_loss < self.best_score - self.min_delta:
            self.best_score = val_loss
            self.best_model_state = copy.deepcopy(model.state_dict())
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

    def restore_best_weights(self, model):
        model.load_state_dict(self.best_model_state)



In [20]:
learning_rate = 0.001
criterion = nn.CrossEntropyLoss() 
optimizer = optim.AdamW(params=vgg16.classifier.parameters(), lr=learning_rate)

In [21]:
early_stopping = EarlyStopping(patience=3, min_delta=0.0001)

epochs = 1000
start_time = time.time()
for epoch in range(epochs):

    vgg16.train()
    total_epoch_loss = 0

    for batch_features, batch_labels in train_loader:
        
        batch_features, batch_labels = batch_features.to(device, non_blocking=True), batch_labels.to(device, non_blocking=True)

        outputs= vgg16(batch_features)

        loss = criterion(outputs, batch_labels)
        
        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

        total_epoch_loss = total_epoch_loss + loss.item()

    avg_loss = total_epoch_loss/len(train_loader)
    print(f'Epoch: {epoch + 1} , Loss: {avg_loss}')

    vgg16.eval()
    val_losses = []
    with torch.no_grad():
        for batch_features, batch_labels in test_loader:

            batch_features, batch_labels = batch_features.to(device, non_blocking=True), batch_labels.to(device, non_blocking=True)

            outputs = vgg16(batch_features)
            loss = criterion(outputs, batch_labels)
            val_losses.append(loss.item())
    
    mean_val_loss = sum(val_losses) / len(val_losses)
    print(f"Epoch {epoch+1}, Validation Loss: {mean_val_loss:.4f}")

    early_stopping(mean_val_loss, model=vgg16)
    if early_stopping.early_stop:
        print("Early stopping triggered! Stopping training.")
        break

early_stopping.restore_best_weights(model=vgg16)

end_time = time.time()
elapsed_seconds = end_time - start_time

hours = int(elapsed_seconds // 3600)
minutes = int((elapsed_seconds % 3600) // 60)
seconds = int(elapsed_seconds % 60)

print(f"Elapsed time: {hours} hours, {minutes} minutes, {seconds} seconds")

Epoch: 1 , Loss: 2.3945855336911133
Epoch 1, Validation Loss: 2.2370
Epoch: 2 , Loss: 2.309909642632328
Epoch 2, Validation Loss: 2.2435
Epoch: 3 , Loss: 2.2968284585582674
Epoch 3, Validation Loss: 2.2772
Epoch: 4 , Loss: 2.2958069437348256
Epoch 4, Validation Loss: 2.9276
Early stopping triggered! Stopping training.
Elapsed time: 0 hours, 35 minutes, 33 seconds


In [22]:
vgg16.eval()
total = 0
correct = 0

for batch_features, batch_labels in test_loader:

    batch_features, batch_labels = batch_features.to(device, non_blocking=True), batch_labels.to(device, non_blocking=True)

    outputs = vgg16(batch_features)
    _, pred = torch.max(outputs, 1)
    total += batch_labels.shape[0]
    correct += (pred == batch_labels).sum().item()

print("Total:", total)
print("Corrected:", correct)
print("Accuracy:", (correct/total) * 100)

Total: 1500
Corrected: 254
Accuracy: 16.933333333333334
